In [1]:
from google.colab import drive
drive.mount('/content/drive')
# % cd drive/My Drive/hw3/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import dataset
import ranking as rnk
import evaluate as evl
import numpy as np
import torch
import matplotlib.pyplot as plt
from scipy import stats
from torch.utils.data import Dataset, TensorDataset, DataLoader
import itertools

import math
data = dataset.get_dataset().get_data_folds()[0]
data.read_data()

In [5]:

#device = 'cuda' if torch.cuda.is_available() else 'cpu'
#torch.manual_seed(0)

class RankNet(torch.nn.Module):
    def __init__(self, n_feature, n_hidden):
        super(RankNet, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)
        self.output = torch.nn.Linear(n_hidden, 1)      
        
    def forward(self, x1):
        x = torch.nn.functional.relu(self.hidden(x1))
        x = self.output(x)
        
        return x

class Model():
    def __init__(self, n_feature, n_hidden, learning_rate):
        self.ranknet = RankNet(n_feature, n_hidden)
        #.to(device)
        self.optimizer = torch.optim.SGD(self.ranknet.parameters(), lr=learning_rate)

def eval_model(model, data_fold):
    #with torch.no_grad():
        x = torch.from_numpy(data_fold.feature_matrix).float()
        y = data_fold.label_vector
        model.ranknet.eval()
               
        output = model.ranknet(x)
        output = output.detach().cpu().numpy().squeeze()
        
        #loss = torch.FloatTensor([0.5])*(torch.FloatTensor([1])-y)*((si-sj).sigmoid()).view(si.size(0))+torch.log(torch.FloatTensor([1])+torch.exp(-((si-sj).sigmoid().view(si.size(0)))))
        scores = evl.evaluate(data_fold, np.asarray(output))  

        return scores


def load_dataset():
    # data = dataset.get_dataset().get_data_folds()[0]
    # data.read_data()

    train_x = torch.from_numpy(data.train.feature_matrix).float()
    train_y = torch.from_numpy(data.train.label_vector).float()

    # documents = data.train.feature_matrix
    # doc_list = list(range(len(documents)))
    
    # # Carthesian product
    # Carth = list(itertools.combinations(doc_list,2))
    # x1, x2, target = [], [], []
  
    # # iterate over all possible combinations
    # for i,j in Carth:
    #     x1.append(docs[i])
    #     x2.append(docs[j])
    #     if data.train.label_vector[i]>data.train.label_vector[j]:
    #         # this is the S_{ij}
    #         target.append(float(1))
    #     elif data.train.label_vector[i]<data.train.label_vector[j]:
    #         target.append(float(-1))
    #     else:
    #         target.append(float(0))


    train_set = TensorDataset(train_x, train_y)
    train_loader = DataLoader(dataset=train_set, batch_size=32, shuffle=True)
#     return torch.FloatTensor(x1), torch.FloatTensor(x2), torch.FloatTensor(target)
    #return data
    return train_loader
       

def plot_ndcg_loss(losses, ndcgs):
    x = np.arange(len(losses))
    fig, ax = plt.subplots()
    
    ax.plot(x, losses, label='Loss')
    ax.plot(x, ndcgs, label='NDCG')
    ax.set_xlabel("Batch % 2000")
    ax.set_ylabel("Score")
    ax.set_title("Pointwise LTR")
    legend = ax.legend(loc='upper center')
    
    plt.show()
    plt.savefig('Pointwise_LTR_plot.png')

    
def train_batch(documentfeatures, labels, model):
#     model.ranknet.train()
    for epoch in range(1):
#         for qid in range(0, train_data.num_queries()):
#             if train_data.query_size(qid) < 2:
#                 continue
 
            model.optimizer.zero_grad()

            output = model.ranknet(documentfeatures)
            
            loss = pairwiseloss(output, labels)
            
            loss.backward()
            
            model.optimizer.step()
            
    return model
    
def pairwiseloss(predictedvals, values):
    
    predictedvals = predictedvals.squeeze()

    # Hier is waar er is gesleuteld aan de data om de nan's te voorkomen
    if predictedvals.shape[0] == 0:
        return torch.tensor([0.0], requires_grad= True)
    # pairs = int(math.factorial(n_docs) / (math.factorial(n_docs - 2) * 2))
    tups = list(itertools.combinations(range(predictedvals.shape[0]), 2))
    

    val1, val2 = [x[0] for x in tups], [x[1] for x in tups]
    #todevice
    pred1 = predictedvals[val1]
    pred2 = predictedvals[val2]
    
    true1 = values[val1]
    true2 = values[val2]
    #.todevice
    l1 = (true1 > true2).type(torch.ByteTensor)
    l2 = (true1 < true2).type(torch.ByteTensor)
    
    S =  l1 - l2
    S = torch.tensor(S)
    sigmoid = torch.sigmoid(pred1.float() - pred2.float())
    C_T = (0.5 * (1 - S) * sigmoid + torch.log(1 + torch.exp(-sigmoid)))
    

    return C_T.mean()

def hyperparam_search():
    # hyper-parameters
    epochs = 10
    learning_rates = [10**-1, 10**-2, 10**-3, 10**-4]
    n_hiddens = [100, 150, 200, 250, 300, 350, 400]
#     learning_rates = [0.001]
#     n_hiddens = [100]
    print("hi")
#     train_loader = load_dataset()
    print("2")
    best_ndcg = 0
    for learning_rate in learning_rates:
        for n_hidden in n_hiddens:
        
            print("\nTesting learning_rate = {} and n_hidden = {}".format(learning_rate, n_hidden))
            model = Model(data.num_features, n_hidden, learning_rate)
            
            last_ndcg = 0
            for epoch in range(epochs):
                
                model.ranknet.train()
                for qid in range(0, data.train.num_queries()):#
                    if data.train.query_size(qid) < 2:#
                        continue#
                    s_i, e_i = data.train.query_range(qid)
            
                    documentfeatures = torch.tensor(data.train.feature_matrix[s_i:e_i]).float()
                    labels = torch.tensor(data.train.label_vector[s_i:e_i])
#                     if documentfeatures.shape[0] == 0:
#                         return torch.tensor([0.0], requires_grad= True)

                    model = train_batch(documentfeatures, labels, model)  

                                       
                scores = eval_model(model, data.validation)
              
                ndcg = scores["ndcg"][0]
                print("Epoch: {}, ndcg: {}".format(epoch, ndcg))
                            
                if ndcg < last_ndcg:
                    break
                last_ndcg = ndcg
                if ndcg > best_ndcg:
                    best_ndcg = ndcg
                    best_params = {"learning_rate": learning_rate, "n_hidden": n_hidden, "epoch": epoch}            
                    print("Best parameters:", best_params)
    
    return best_params

hyperparam_search()



#### Loss functie 1tje kiezen voor validation en train.
####  

# {'learning_rate': 0.1, 'n_hidden': 100, 'epoch': 8}


hi
2

Testing learning_rate = 0.1 and n_hidden = 100


/Users/catlinbruys/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:129: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


"metric": "mean" ("standard deviation")
Epoch: 0, ndcg: 0.8339711885799135
Best parameters: {'learning_rate': 0.1, 'n_hidden': 100, 'epoch': 0}
"metric": "mean" ("standard deviation")
Epoch: 1, ndcg: 0.836585835033126
Best parameters: {'learning_rate': 0.1, 'n_hidden': 100, 'epoch': 1}
"metric": "mean" ("standard deviation")
Epoch: 2, ndcg: 0.8385919713570898
Best parameters: {'learning_rate': 0.1, 'n_hidden': 100, 'epoch': 2}
"metric": "mean" ("standard deviation")
Epoch: 3, ndcg: 0.8405189486595361
Best parameters: {'learning_rate': 0.1, 'n_hidden': 100, 'epoch': 3}
"metric": "mean" ("standard deviation")
Epoch: 4, ndcg: 0.8423262520359234
Best parameters: {'learning_rate': 0.1, 'n_hidden': 100, 'epoch': 4}
"metric": "mean" ("standard deviation")
Epoch: 5, ndcg: 0.8431964399239792
Best parameters: {'learning_rate': 0.1, 'n_hidden': 100, 'epoch': 5}
"metric": "mean" ("standard deviation")
Epoch: 6, ndcg: 0.842429820681789

Testing learning_rate = 0.1 and n_hidden = 150
"metric": "mean

"metric": "mean" ("standard deviation")
Epoch: 5, ndcg: 0.8399213375092767
"metric": "mean" ("standard deviation")
Epoch: 6, ndcg: 0.8403784165518466
"metric": "mean" ("standard deviation")
Epoch: 7, ndcg: 0.8408707925123406
"metric": "mean" ("standard deviation")
Epoch: 8, ndcg: 0.8409646609188093
"metric": "mean" ("standard deviation")
Epoch: 9, ndcg: 0.841470012972615

Testing learning_rate = 0.01 and n_hidden = 350
"metric": "mean" ("standard deviation")
Epoch: 0, ndcg: 0.83054144617121
"metric": "mean" ("standard deviation")
Epoch: 1, ndcg: 0.8339794009094857
"metric": "mean" ("standard deviation")
Epoch: 2, ndcg: 0.8358795990295078
"metric": "mean" ("standard deviation")
Epoch: 3, ndcg: 0.8370064568651575
"metric": "mean" ("standard deviation")
Epoch: 4, ndcg: 0.8381256182533403
"metric": "mean" ("standard deviation")
Epoch: 5, ndcg: 0.8388627432079988
"metric": "mean" ("standard deviation")
Epoch: 6, ndcg: 0.8392070080438895
"metric": "mean" ("standard deviation")
Epoch: 7, ndcg

"metric": "mean" ("standard deviation")
Epoch: 0, ndcg: 0.7702173509077193
"metric": "mean" ("standard deviation")
Epoch: 1, ndcg: 0.7892815564393412
"metric": "mean" ("standard deviation")
Epoch: 2, ndcg: 0.7988967992001363
"metric": "mean" ("standard deviation")
Epoch: 3, ndcg: 0.8033685314099449
"metric": "mean" ("standard deviation")
Epoch: 4, ndcg: 0.8070413254837143
"metric": "mean" ("standard deviation")
Epoch: 5, ndcg: 0.8088839480261145
"metric": "mean" ("standard deviation")
Epoch: 6, ndcg: 0.8104182933309487
"metric": "mean" ("standard deviation")
Epoch: 7, ndcg: 0.8107388963825832
"metric": "mean" ("standard deviation")
Epoch: 8, ndcg: 0.8121125071572319
"metric": "mean" ("standard deviation")
Epoch: 9, ndcg: 0.8125482690084458

Testing learning_rate = 0.0001 and n_hidden = 200
"metric": "mean" ("standard deviation")
Epoch: 0, ndcg: 0.7928923825483036
"metric": "mean" ("standard deviation")
Epoch: 1, ndcg: 0.8039486983661346
"metric": "mean" ("standard deviation")
Epoch: 2,

{'learning_rate': 0.1, 'n_hidden': 250, 'epoch': 6}

In [6]:

def train_best(best_params):
    epochs = best_params["epoch"]
    n_hidden = best_params["n_hidden"]
    learning_rate = best_params["learning_rate"]
    
    # load data
#     data, train_loader = load_dataset()
    model = Model(data.num_features, n_hidden, learning_rate)

    losses, ndcgs = [], []
    for epoch in range(epochs):
        eval_count = 0
        for qid in range(0, data.train.num_queries()):#
            if data.train.query_size(qid) < 2:#
                continue#
            s_i, e_i = data.train.query_range(qid)
            
            documentfeatures = torch.tensor(data.train.feature_matrix[s_i:e_i]).float()
            labels = torch.tensor(data.train.label_vector[s_i:e_i])
            model = train_batch(documentfeatures, labels, model) 
            eval_count +=1
            if eval_count % 2000 == 0:
                loss, scores = eval_model(model, data.validation)
                losses.append(loss)
                ndcgs.append(scores["ndcg"][0])
        print("Epoch: {}, ndcg: {}".format(epoch, scores["ndcg"][0]))
        
    return ndcgs, losses, model

In [ ]:

# def train_best(best_params):
#     epochs = best_params["epoch"]
#     n_hidden = best_params["n_hidden"]
#     learning_rate = best_params["learning_rate"]
    
#     # load data
#     data, train_loader = load_dataset()
#     model = Model(data.num_features, n_hidden, learning_rate)

#     losses, ndcgs = [], []
#     for epoch in range(epochs):
#         eval_count = 0
#         for x_batch, y_batch in train_loader:
#             model = train_batch(x_batch, y_batch, model)
#             eval_count +=1
#             if eval_count % 2000 == 0:
#                 loss, scores = eval_model(model, data.validation)
#                 losses.append(loss)
#                 ndcgs.append(scores["ndcg"][0])
#         print("Epoch: {}, ndcg: {}".format(epoch, scores["ndcg"][0]))
        
#     return ndcgs, losses, model


def get_distributions(model):
    data = dataset.get_dataset().get_data_folds()[0]
    data.read_data()
    model.ranknet.eval()

    val_x = torch.from_numpy(data.validation.feature_matrix).float().to(device)
    test_x = torch.from_numpy(data.test.feature_matrix).float().to(device)
           
    val = model.ranknet(val_x).detach().cpu().numpy().squeeze()
    test = model.ranknet(test_x).detach().cpu().numpy().squeeze()
    actual = np.concatenate((data.train.label_vector, data.validation.label_vector, data.test.label_vector))
    
    distributions = {
    "val_mean": np.mean(val),
    "val_std": np.std(val),
    "test_mean": np.mean(test),
    "test_std": np.std(test),
    "actual_mean": np.mean(actual), 
    "actual_std": np.std(actual),
    }
    
    return distributions



if __name__ == "__main__":
    #determine best hyper parameters
    best_params = hyperparam_search()
    #train best model
    ndcgs, losses, model = train_best(best_params)
    #plot ndcg and loss    
    plot_ndcg_loss(losses, ndcgs)
    #get distributions of scores
    distributions = get_distributions(model)
    #performance on test set
    data = dataset.get_dataset().get_data_folds()[0]
    data.read_data()
    loss, scores = eval_model(model, data.test)



hi
2

Testing learning_rate = 0.1 and n_hidden = 100


/Users/catlinbruys/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:129: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


"metric": "mean" ("standard deviation")
Epoch: 0, ndcg: 0.8342661932641088
Best parameters: {'learning_rate': 0.1, 'n_hidden': 100, 'epoch': 0}
"metric": "mean" ("standard deviation")
Epoch: 1, ndcg: 0.8380042037604613
Best parameters: {'learning_rate': 0.1, 'n_hidden': 100, 'epoch': 1}
"metric": "mean" ("standard deviation")
Epoch: 2, ndcg: 0.8397042115047316
Best parameters: {'learning_rate': 0.1, 'n_hidden': 100, 'epoch': 2}
"metric": "mean" ("standard deviation")
Epoch: 3, ndcg: 0.841438670211334
Best parameters: {'learning_rate': 0.1, 'n_hidden': 100, 'epoch': 3}
"metric": "mean" ("standard deviation")
Epoch: 4, ndcg: 0.841388114939132

Testing learning_rate = 0.1 and n_hidden = 150
"metric": "mean" ("standard deviation")
Epoch: 0, ndcg: 0.8348390950434817
"metric": "mean" ("standard deviation")
Epoch: 1, ndcg: 0.8376546130755324
"metric": "mean" ("standard deviation")
Epoch: 2, ndcg: 0.8406267082843206
"metric": "mean" ("standard deviation")
Epoch: 3, ndcg: 0.8414030421801957
"me

"metric": "mean" ("standard deviation")
Epoch: 8, ndcg: 0.840719278074929
"metric": "mean" ("standard deviation")
Epoch: 9, ndcg: 0.8412214656486979

Testing learning_rate = 0.01 and n_hidden = 350
"metric": "mean" ("standard deviation")
Epoch: 0, ndcg: 0.8318629954032453
"metric": "mean" ("standard deviation")
Epoch: 1, ndcg: 0.8343491319352953
"metric": "mean" ("standard deviation")
Epoch: 2, ndcg: 0.8359726556861994
"metric": "mean" ("standard deviation")
Epoch: 3, ndcg: 0.8369390553993352
"metric": "mean" ("standard deviation")
Epoch: 4, ndcg: 0.8376904487608666
"metric": "mean" ("standard deviation")
Epoch: 5, ndcg: 0.8387170475682597
"metric": "mean" ("standard deviation")
Epoch: 6, ndcg: 0.839838594512447


In [ ]:

#device = 'cuda' if torch.cuda.is_available() else 'cpu'
#torch.manual_seed(0)

class RankNetSU(torch.nn.Module):
    def __init__(self, n_feature, n_hidden):
        super(RankNet, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)
        self.output = torch.nn.Linear(n_hidden, 1)      
        
    def forward(self, x1):
        x = torch.nn.functional.relu(self.hidden(x1))
        x = self.output(x)
        
        return x

class Model():
    def __init__(self, n_feature, n_hidden, learning_rate):
        self.ranknet = RankNetSU(n_feature, n_hidden)
        #.to(device)
        self.optimizer = torch.optim.SGD(self.ranknet.parameters(), lr=learning_rate)

def eval_model(model, data_fold):
    #with torch.no_grad():
        x = torch.from_numpy(data_fold.feature_matrix).float()
        y = data_fold.label_vector
        model.ranknet.eval()
               
        output = model.ranknet(x)
        output = output.detach().cpu().numpy().squeeze()
        
        #loss = torch.FloatTensor([0.5])*(torch.FloatTensor([1])-y)*((si-sj).sigmoid()).view(si.size(0))+torch.log(torch.FloatTensor([1])+torch.exp(-((si-sj).sigmoid().view(si.size(0)))))
        scores = evl.evaluate(data_fold, np.asarray(output))  

        return scores


def load_dataset():

    train_x = torch.from_numpy(data.train.feature_matrix).float()
    train_y = torch.from_numpy(data.train.label_vector).float()

    train_set = TensorDataset(train_x, train_y)
    train_loader = DataLoader(dataset=train_set, batch_size=32, shuffle=True)

    return train_loader
       

def plot_ndcg_loss(losses, ndcgs):
    x = np.arange(len(losses))
    fig, ax = plt.subplots()
    
    ax.plot(x, losses, label='Loss')
    ax.plot(x, ndcgs, label='NDCG')
    ax.set_xlabel("Batch % 2000")
    ax.set_ylabel("Score")
    ax.set_title("Pointwise LTR")
    legend = ax.legend(loc='upper center')
    
    plt.show()
    plt.savefig('Pointwise_LTR_plot.png')

    
def train_batch(documentfeatures, labels, model):
#     model.ranknet.train()
    for epoch in range(1):
#         for qid in range(0, train_data.num_queries()):
#             if train_data.query_size(qid) < 2:
#                 continue
 
            model.optimizer.zero_grad()

            output = model.ranknet(documentfeatures)
            
            loss = pairwiseloss(output, labels)
            break
            
            loss.backward()
            
            model.optimizer.step()
            
    return model
    
def pairwiseloss(predictedvals, values):
    
    predictedvals = predictedvals.squeeze()

    if predictedvals.shape[0] == 0:
        return torch.tensor([0.0], requires_grad= True)
    tups = list(itertools.combinations(range(predictedvals.shape[0]), 2))
    

    val1, val2 = [x[0] for x in tups], [x[1] for x in tups]
    #todevice
    pred1 = predictedvals[val1]
    pred2 = predictedvals[val2]
    
    true1 = values[val1]
    true2 = values[val2]
    #.todevice
    l1 = (true1 > true2).type(torch.ByteTensor)
    l2 = (true1 < true2).type(torch.ByteTensor)
    
    S =  l1 - l2
    S = torch.tensor(S)
    sigmoid = torch.sigmoid(pred1.float() - pred2.float())
#   C_T = (0.5 * (1 - S) * sigmoid + torch.log(1 + torch.exp(-sigmoid)))
    lambda_ij = (torch.FloatTensor([0.5])*(torch.FloatTensor([1])-S)-(torch.FloatTensor([1])/(torch.FloatTensor([1])+sigmoid))).sigmoid()
    print(lambda_ij.shape)
#     lambda_ij = torch.sigmoid(torch.FloatTensor([0.5])*(torch.FloatTensor([1])-S)torch.log(torch.FloatTensor([1])+torch.exp(-((s_i-s_j).sigmoid().view(s_i.size(0)))))

    return lambda_ij.sum()

def hyperparam_search():
    # hyper-parameters
    epochs = 10
    learning_rates = [10**-1, 10**-2, 10**-3, 10**-4]
    n_hiddens = [100, 150, 200, 250, 300, 350, 400]
#     learning_rates = [0.001]
#     n_hiddens = [100]
    print("hi")
#     train_loader = load_dataset()
    print("2")
    best_ndcg = 0
    for learning_rate in learning_rates:
        for n_hidden in n_hiddens:
        
            print("\nTesting learning_rate = {} and n_hidden = {}".format(learning_rate, n_hidden))
            model = Model(data.num_features, n_hidden, learning_rate)
            
            last_ndcg = 0
            for epoch in range(epochs):
                
                model.ranknet.train()
                for qid in range(0, data.train.num_queries()):#
                    if data.train.query_size(qid) < 2:#
                        continue#
                    s_i, e_i = data.train.query_range(qid)
            
                    documentfeatures = torch.tensor(data.train.feature_matrix[s_i:e_i]).float()
                    labels = torch.tensor(data.train.label_vector[s_i:e_i])
#                     if documentfeatures.shape[0] == 0:
#                         return torch.tensor([0.0], requires_grad= True)

                    model = train_batch(documentfeatures, labels, model)  

                                       
                scores = eval_model(model, data.validation)
              
                ndcg = scores["ndcg"][0]
                print("Epoch: {}, ndcg: {}".format(epoch, ndcg))
                            
                if ndcg < last_ndcg:
                    break
                last_ndcg = ndcg
                if ndcg > best_ndcg:
                    best_ndcg = ndcg
                    best_params = {"learning_rate": learning_rate, "n_hidden": n_hidden, "epoch": epoch}            
                    print("Best parameters:", best_params)
    
    return best_params

hyperparam_search()



#### Loss functie 1tje kiezen voor validation en train.
####  

# {'learning_rate': 0.1, 'n_hidden': 100, 'epoch': 8}



In [0]:
from scipy import stats
sped_ndcg, sped_arr = [], []
class SpedUpRanknet():
  def __init__(self, num_features, hidden_units=10, device="cpu", lr = 0.05, layers = 1,  early_stopping=15): #, encoder, decoder, classes, n_protos, bottleneck_size=40, device='cpu', model_type='simple'
    '''
    arguments:
      num_features = size of input vector (features size)
    '''
    self.normal_ndcg = = [0 for _ in range(early_stopping)]
    self.num_features = num_features
    self.lr = lr
    assert layers in [1,2,3,4], "layers need to be 1,2,3 or 4"
    if layers == 2:
      self.model = nn.Sequential(
              nn.Linear(num_features, hidden_units),
              nn.Dropout(0.5),
              nn.BatchNorm1d(hidden_units),
              nn.LeakyReLU(0.2, inplace=True),
              nn.Linear(hidden_units, 1)
          )
    elif layers == 1:
      self.model = nn.Sequential(
              nn.Linear(num_features, 1)
          )
    elif layers == 3:
      self.model = nn.Sequential(
              nn.Linear(num_features, int(hidden_units/2)),
              nn.Dropout(0.5),
              nn.BatchNorm1d(int(hidden_units/2)),
              nn.LeakyReLU(0.2, inplace=True),
              nn.Linear(int(hidden_units/2), hidden_units),
              nn.Dropout(0.5),
              nn.BatchNorm1d(hidden_units),
              nn.LeakyReLU(0.2, inplace=True),
              nn.Linear(hidden_units, 1)
          )
    elif layers == 4:
      self.model = nn.Sequential(
              nn.Linear(num_features, int(hidden_units/2)),
              nn.Dropout(0.5),
              nn.BatchNorm1d(int(hidden_units/2)),
              nn.LeakyReLU(0.2, inplace=True),
              nn.Linear(int(hidden_units/2), hidden_units),
              nn.Dropout(0.5),
              nn.BatchNorm1d(hidden_units),
              nn.LeakyReLU(0.2, inplace=True),
              nn.Linear(hidden_units, int(hidden_units/2)),
              nn.Dropout(0.5),
              nn.BatchNorm1d(int(hidden_units/2)),
              nn.LeakyReLU(0.2, inplace=True),
              nn.Linear(int(hidden_units/2), 1)
          )


    self.output_sig = nn.Sigmoid()
    self.device=device
    self.model.to(device)
    for mod in self.model:
      if isinstance(mod, nn.Linear):
        nn.init.normal_(mod.weight, mean=2, std=2)
        mod.bias.data.fill_(0)
  
  def forward(self, input_1):
    '''
    Does one forward pass using two inputs
    '''
    si = self.model(input_1.float())
    # sj = self.model(input_2.float())#self.output_sig(x)
    
    return si

  # def get_feature_vectors_target_train(self, s_i, e_i):
  #   docs = data.train.feature_matrix[s_i:e_i]
  #   document_list = list(range(len(docs)))

  #   # create dict from idx of list location to the feature vector
  #   idx_2_fm = {}
  #   for e, d in enumerate(docs):
  #     idx_2_fm[e] = d

  #   # create all the possible combinations
  #   doc_combs_in_qid = list(itertools.combinations(document_list,2))
  #   input_1, input_2, target = [], [], []
  
  #   # iterate over all possible combinations
  #   for i,j in doc_combs_in_qid:
  #     input_1.append(docs[i])
  #     input_2.append(docs[j])
  #     if data.train.label_vector[i+s_i]>data.train.label_vector[j+s_i]:
  #       # this is the S_{ij}
  #       target.append(float(1))
  #     elif data.train.label_vector[i+s_i]<data.train.label_vector[j+s_i]:
  #       target.append(float(-1))
  #     else:
  #       target.append(float(0))
  #   return torch.FloatTensor(input_1), torch.FloatTensor(input_2), torch.FloatTensor(target)

  def fast_loss(self, forward_output, first_doc, last_doc):
    # docs = data.train.feature_matrix[s_i:e_i]
    document_list = list(range(last_doc-first_doc))

    # idx_2_fm = {}
    # for e, d in enumerate(docs):
    #   idx_2_fm[e] = d

    doc_combs_in_qid = list(itertools.combinations(document_list,2))
    q_loss = torch.zeros(1) + (float(0)*forward_output[0])
    for i,j in doc_combs_in_qid:
      s_i = forward_output[i]
      s_j = forward_output[j]
      if data.train.label_vector[i+first_doc]>data.train.label_vector[j+first_doc]:
        # this is the S_{ij}
        S_ij = float(1)
      elif data.train.label_vector[i+first_doc]<data.train.label_vector[j+first_doc]:
        S_ij = float(-1)
      else:
        continue
      loss = torch.FloatTensor([0.5])*(torch.FloatTensor([1])-S_ij)*((s_i-s_j).sigmoid()).view(s_i.size(0))+torch.log(torch.FloatTensor([1])+torch.exp(-((s_i-s_j).sigmoid().view(s_i.size(0)))))

      # loss = (torch.FloatTensor([0.5])*(torch.FloatTensor([1])-S_ij)-(torch.FloatTensor([1])/(torch.FloatTensor([1])+torch.exp((s_i-s_j).sigmoid().view(s_i.size(0)))))).sigmoid()
      q_loss += loss
    # print(q_loss)
    return q_loss

  def train(self, n_epochs):
    lr=self.lr
    if not os.path.exists('loss_folder'):
        os.makedirs('loss_folder')

    optimizer = optim.Adam(self.model.parameters(), lr=lr)

    losses = {'Ranking_class': [] }
    accuracies = {'Test': [], 'Train': [] }
    # data arguments: ['datafold', 'doc_feat', 'doc_str', 'doclist_ranges', 'feature_matrix', 'label_vector', 'name', 'num_docs', 'num_queries', 'query_feat', 'query_labels', 'query_range', 'query_size', 'query_sizes']
    for epoch in range(n_epochs):
        c_loss = []
        train_acc = []
        
        num_queries = list(range(1, data.train.num_queries()))
        # if epoch != 0:
        #   random.shuffle(num_queries)
        # num_queries = num_queries[0 : int(len(num_queries) * early_stop)]
        for qid in num_queries:
            s_i, e_i = data.train.query_range(qid)

            # input_1, input_2, target = self.get_feature_vectors_target_train(s_i, e_i)

            optimizer.zero_grad()
            input_1 = torch.FloatTensor(data.train.feature_matrix[s_i:e_i])
            try:
              si = self.forward(input_1)
            except:
              print("Query with one document, skipping this query", qid)
              continue

            loss = self.fast_loss(si, s_i, e_i)

            # loss = torch.FloatTensor([0.5])*(torch.FloatTensor([1])-target)*((si-sj).sigmoid()).view(si.size(0))+torch.log(torch.FloatTensor([1])+torch.exp(-((si-sj).sigmoid().view(si.size(0)))))
            # loss = (torch.FloatTensor([0.5])*(torch.FloatTensor([1])-target)-(torch.FloatTensor([1])/(torch.FloatTensor([1])+torch.exp((si-sj).sigmoid().view(si.size(0)))))).sigmoid()

            loss.sum().backward()
            optimizer.step()

            if not qid%100:
              print(f"Data: {qid}, with a loss of {loss.sum()}")
              all_scores = self.model(torch.FloatTensor(data.validation.feature_matrix)).view(-1).detach().numpy()
              metrics = self.evaluate(data.validation, all_scores)
              ndcg = metrics['ndcg']
              arr = metrics['relevant rank']
              sped_ndcg.append(ndcg)
              sped_arr.append(arr)
              print(f"the ndcg is {ndcg} the arr is {arr}")
              self.normal_ndcg.append(ndcg[0])
              self.normal_ndcg.pop(0)
              x = list(range(0, len(self.normal_ndcg)))
              slope, *_ = stats.linregress(x, self.normal_ndcg)
              if slope < 0:
                print(f"negative slope, early stopping activated slope of: {slope}")
                break

    all_scores = self.model(torch.FloatTensor(data.train.feature_matrix)).view(-1).detach().numpy()
    metrics = self.evaluate(data.train, all_scores)
    ndcg = metrics['ndcg']
    print(f"Train ndcg is {ndcg}")

    # create_log(losses, accuracies, log_file)
    # torch.save(model.state_dict(), os.path.join(model_folder, 'model_final.pt'))

  def predict(self, data_split):
      # self.eval()
      allscores = []
      x_val_tensor = torch.from_numpy(data_split.feature_matrix).float()
      y_val_tensor = torch.from_numpy(data_split.label_vector)
      
      count = 0 
      for val, label in zip(x_val_tensor, y_val_tensor):
          y_pred = self.model(val)
          allscores.append(y_pred.numpy())
          
      return np.array(allscores)

  def evaluate(self, data_split, all_scores):
    '''
    function to evaluate the model
    '''
    metrics = evl.evaluate(data_split, all_scores, False)
    return metrics

sped_model = SpedUpRanknet(num_features=data.num_features, hidden_units=10, lr = 0.05, layers = 1, device='cpu')
# print("hier")
sped_model.train(n_epochs=1)